In [64]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\box_square_functions.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2,
GLMakie, LinearAlgebra, LinearSolve

pathtofile = "C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab\\"
I0_hom = load(pathtofile * "I0_hom_hom.jld")["data"]
u0_hom = load(pathtofile * "U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [65]:
index_point_from_curve = 1
t = 2000.0; tspan = [0.0, t]
ϵ_box = 1.0e-5; ϵ_shift = 1.0e-3
E = interval(-40, 40); x = interval(-5, 5); y = interval(-5, 5)
box = IntervalBox(E, x, y);

In [66]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305
U0 = u0_hom[index_point_from_curve]; I0 = I0_hom[index_point_from_curve]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0];

In [67]:
count_points_on_side = 101

101

In [68]:
function get_norm_αs_arrays(points, fp, A)

    count_rows, count_cols = size(points)
    index = 1
    last_index = count_rows
    matrix_αs = zeros(count_rows, count_cols)
    array_norms = zeros(count_rows)

    while index <= last_index

        norm_, αs = get_norm_αs(points[index, :], fp, A)
        matrix_αs[index, :] = αs
        array_norms[index] = norm_
        index+=1
        
    end
    
    return array_norms, matrix_αs
end
function get_norm_αs(u, fp, A)
    x = Vector(u)
    b = x - fp
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end

function get_fp(system, jac_system, p, box, dim)

    u0 = zeros(dim)
    ds = CoupledODEs(system, u0, p)
    fp, _, _ = fixedpoints(ds, box, jac_system)

    return fp
end
function get_matrix(fp, p, jac_system, norm = 2)

    Jfp = jac_system(fp, p, 0.0)
    eigen_values_vectors = eigen(Jfp)
    eigen_vectors = eigen_values_vectors.vectors

    v1 = real(eigen_vectors[:, 1])
    v2 = real(eigen_vectors[:,2])
    v3 = imag(eigen_vectors[:, 3])

    v1 = normalize(v1, norm)
    v2 = normalize(v2, norm)
    v3 = normalize(v3, norm)

    v1 = transpose(v1)
    v2 = transpose(v2)
    v3 = transpose(v3)

    A = transpose([v1; v2; v3])

    return A
end;

In [69]:
function side_square_left(ϵ_box, count_points_on_side, fp, A, dim)
    matrix_point = zeros(count_points_on_side, dim)
    number_point = 1
    α1 = 0.0
    α2 = -ϵ_box
    while number_point <= (count_points_on_side)
        α3 = - ϵ_box + 2 * ϵ_box * number_point / count_points_on_side
        α_vector = [α1, α2, α3]
        point_on_side_square = fp + A * α_vector
        matrix_point[number_point, :] = point_on_side_square
        number_point+=1
    end
    return matrix_point
end

function side_square_right(ϵ_box, count_points_on_side, fp, A, dim)
    matrix_point = zeros(count_points_on_side, dim)
    number_point = 1
    α1 = 0.0
    α2 = ϵ_box
    while number_point <= (count_points_on_side)
        α3 = - ϵ_box + 2 * ϵ_box * number_point / count_points_on_side
        α_vector = [α1, α2, α3]
        point_on_side_square = fp + A * α_vector
        matrix_point[number_point, :] = point_on_side_square
        number_point+=1
    end
    return matrix_point
end
function side_square_up(ϵ_box, count_points_on_side, fp, A, dim)
    matrix_point = zeros(count_points_on_side, dim)
    number_point = 1
    α1 = 0.0
    α3 = ϵ_box
    while number_point <= (count_points_on_side)
        α2 = - ϵ_box + 2 * ϵ_box * number_point / count_points_on_side
        α_vector = [α1, α2, α3]
        point_on_side_square = fp + A * α_vector
        matrix_point[number_point, :] = point_on_side_square
        number_point+=1
    end
    return matrix_point
end
function side_square_down(ϵ_box, count_points_on_side, fp, A, dim)
    matrix_point = zeros(count_points_on_side, dim)
    number_point = 1
    α1 = 0.0
    α3 = -ϵ_box
    while number_point <= (count_points_on_side)
        α2 = - ϵ_box + 2 * ϵ_box * number_point / count_points_on_side
        α_vector = [α1, α2, α3]
        point_on_side_square = fp + A * α_vector
        matrix_point[number_point, :] = point_on_side_square
        number_point+=1
    end
    return matrix_point
end;

In [70]:
fp = get_fp(TM, jacob_TM_, p, box, 3)
A = get_matrix(fp[1], p, jacob_TM_, Inf)
points_up_side = side_square_up(ϵ_box, count_points_on_side, fp[1], A,  3)
points_down_side = side_square_down(ϵ_box, count_points_on_side, fp[1], A,  3)
points_right_side = side_square_right(ϵ_box, count_points_on_side, fp[1], A, 3)
points_left_side = side_square_left(ϵ_box, count_points_on_side, fp[1], A, 3)

normsup, αsup=  get_norm_αs_arrays(points_up_side, fp[1], A)
normsdown, αsodwn =  get_norm_αs_arrays(points_down_side, fp[1], A)

normsleft, αsleft=  get_norm_αs_arrays(points_left_side, fp[1], A)
normsright, αsright =  get_norm_αs_arrays(points_right_side, fp[1], A);

In [ ]:
idx, idy = 2, 3

f = Figure(resolution = (500, 500))
ax = Axis(f[1, 1])

scatter!(αsup[:, idx], αsup[:, idy],
markersize = 5.0, color = :deeppink)
scatter!(αsodwn[:, idx], αsodwn[:, idy],
markersize = 5.0)

scatter!(αsleft[:, idx], αsleft[:, idy],
markersize = 5.0)
#scatter!(αsright[:, idx], αsright[:, idy],
#markersize = 5.0)

scatter!(0.0, 0.0, color = :blue, markersize = 5.0)

display(GLMakie.Screen(), f);